NOTE: To work with any of our notebooks on Deepnote, you must first
- "Duplicate" the project into your own Deepnote account (see the button in the top right of the GUI), then 
- click "Move this file to notebooks".  

Then it should run!

# Writing (simple) systems in Drake

Drake is a powerful modeling language for authoring models of dynamical systems.  The goal of this exercise is to learn how to author a very simple dynamical systems.  To start, take a few minutes to work through [this Drake tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/%2Fdynamical_systems.ipynb).

Your first task is to write a simple *continuous-time* system that implements the dynamics of a simple damped pendulum (with mass, length, and damping set to 1, and gravity set to 10): $$\ddot\theta + \dot\theta + 10\sin\theta = u.$$  Write code into the following block that results in the pendulum_system variable being an *instance* of a Drake System that implements these dynamics using position and velocity as the state.

Hint: You can import `sin` from `numpy` or from `pydrake.math`. Depending on your version of `numpy`, it might print a warning, but this can be safely ignored.

In [19]:
from pydrake.systems.framework import LeafSystem
from pydrake.systems.framework import BasicVector
from pydrake.systems.framework import ContinuousState
from numpy import sin

class SimplePendulumSystem(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        
        # Declare one input port for external torque 'u'.
        self.DeclareVectorInputPort("input_torque", BasicVector(1))

        # Declare continuous state: [theta, thetadot]
        self.DeclareContinuousState(2)  # Two states: theta and thetadot (velocity)

        # Declare one output port, the full state [theta, thetadot]. BasicVector() is an array with 2 elements
        self.DeclareVectorOutputPort("state_output", BasicVector(2),
                                     self.CopyStateOut)

    # Output method: just copy the state to the output port
    def CopyStateOut(self, context, output):
        output.SetFrom(context.get_continuous_state_vector())

    # Time derivatives method: implements the dynamics
    def DoCalcTimeDerivatives(self, context, derivatives):
        state = context.get_continuous_state_vector().CopyToVector()
        theta = state[0]
        theta_dot = state[1]
        
        # Get the input torque 'u'
        u = self.get_input_port(0).Eval(context)[0]
        
        # Define the dynamics
        theta_dot_dot = u - theta_dot - 10 * sin(theta)
        
        # Set the derivatives
        derivatives.get_mutable_vector().SetAtIndex(0, theta_dot)  # dtheta/dt = theta_dot
        derivatives.get_mutable_vector().SetAtIndex(1, theta_dot_dot)  # dthetadot/dt = theta_dot_dot


# Instantiate the System
pendulum_system = SimplePendulumSystem()

Drake [Systems](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system.html) have methods that define the dynamics, like $$\dot{x} = f(t, x, u), \qquad y = g(t, x, u)$$ for the state dynamics, $f$, and system output function, $g$. Sometimes the equations depend on time, sometimes not. Sometimes they have extra parameters, sometimes not. Not all systems have inputs, and not all systems have states!

To make the input arguments to these methods more succinct, we collect $t, x, u$ and parameters into a single structure, which we call the [Context](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_context.html). Then all of the methods for the dynamical system can be called using that context, e.g.: $$\dot{x} = f(\text{context}), \qquad y = g(\text{context}).$$ We'll use it often throughout the course.

In [20]:
if pendulum_system is not None:
    context = pendulum_system.CreateDefaultContext()
    print(context)

    # Set the time and state
    context.SetTime(4.67)
    context.SetContinuousState([2.1, 3.4])
    print(context)

    # Set an input port to have a fixed value
    # (alternatively, we could connect it to the output port of another system).
    pendulum_system.get_input_port(0).FixValue(context, [7.89])
    print(f"Input port 0 value: {pendulum_system.get_input_port(0).Eval(context)}")

::_ Context
------------
Time: 0
States:
  2 continuous states
    0 0


::_ Context
------------
Time: 4.67
States:
  2 continuous states
    2.1 3.4


Input port 0 value: [7.89]


## Autograding
You can check your work by running the following cell:

In [21]:
from underactuated.exercises.grader import Grader
from underactuated.exercises.intro.test_drake_systems import TestDrakeSystems

Grader.grade_output([TestDrakeSystems], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 2/2.

Score for test_dynamics (underactuated.exercises.intro.test_drake_systems.TestDrakeSystems) is 1/1.

Score for test_input_and_state (underactuated.exercises.intro.test_drake_systems.TestDrakeSystems) is 1/1.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=39eb1969-1a41-454c-9463-1e187e5caeeb' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>